In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import comb

In [ ]:
# ------------------------------------------------------------
# Bernstein basis
# ------------------------------------------------------------
def bernstein_poly(n, i, t):
    """Bernstein polynomial B_i^n(t)."""
    return comb(n, i) * (t**i) * ((1 - t) ** (n - i))


def bernstein_basis_matrix(n, t_values):
    """
    Full Bernstein basis matrix:
    rows correspond to t, columns to i.
    shape = (len(t_values), n+1)
    """
    t_values = np.asarray(t_values, dtype=float)
    B = np.zeros((len(t_values), n + 1))
    for i in range(n + 1):
        B[:, i] = [bernstein_poly(n, i, float(t)) for t in t_values]
    return B


# ------------------------------------------------------------
# Bézier curve evaluation using Bernstein basis
# ------------------------------------------------------------
def bezier_eval(control_points, t_values):
    """
    Evaluate Bézier curve points using Bernstein basis.
    control_points: (n+1, 2)
    t_values: array in [0,1]
    """
    P = np.asarray(control_points, dtype=float)
    n = len(P) - 1
    B = bernstein_basis_matrix(n, t_values)  # (m, n+1)
    C = B @ P                                # (m, 2)
    return C, B

In [ ]:
# ------------------------------------------------------------
# Example control points (edit freely)
# ------------------------------------------------------------
cps = np.array([
    [0.0, 0.0],
    [3.0, 2.0],
    [6.0, 2.0],
    [8.0, 0.0],
])

In [ ]:
n = len(cps) - 1
t_curve = np.linspace(0, 1, 250)

curve_pts, B_curve = bezier_eval(cps, t_curve)

# ------------------------------------------------------------
# Select a parameter value t_sel
# ------------------------------------------------------------
t_sel = 0.30

# Compute Bernstein weights at t_sel and the curve point
B_sel = np.array([bernstein_poly(n, i, t_sel) for i in range(n + 1)])
C_sel = B_sel @ cps

# Find closest index in sampled curve for before/after split
idx_sel = np.argmin(np.abs(t_curve - t_sel))


In [ ]:
# ------------------------------------------------------------
# Consistent palette 
# ------------------------------------------------------------
level_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]
palette = level_colors[:n+1]


# ============================================================
# PLOT 1: Bézier curve with "before/after" styling
# ============================================================
plt.figure(figsize=(12, 7))

# Curve after (lighter)
plt.plot(curve_pts[idx_sel:, 0], curve_pts[idx_sel:, 1], alpha=0.2, linewidth=2, label="Curve after $t_{sel}$")

# Curve before (full opacity)
plt.plot(curve_pts[:idx_sel+1, 0], curve_pts[:idx_sel+1, 1], linewidth=2, label="Curve before $t_{sel}$")

# Control polygon
plt.plot(cps[:, 0], cps[:, 1], "o--", color="red", label="Control polygon")

# Selected point
plt.scatter(C_sel[0], C_sel[1], color="red", s=130, zorder=5, label=f"$C(t)$ at t={t_sel:.2f}")

plt.title("Bézier Curve — before/after styling and selected point")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.legend()
plt.show()


# ============================================================
# PLOT 2: Bernstein basis functions + values at t_sel
# ============================================================
plt.figure(figsize=(12, 7))

for i in range(n + 1):
    plt.plot(t_curve, B_curve[:, i], color=palette[i], linewidth=2, label=f"$B_{{{i}}}^{n}(t)$")

    # Marker at t_sel
    plt.scatter(t_sel, B_sel[i], color=palette[i], s=80, zorder=5)

    # Label next to marker
    plt.text(t_sel + 0.015, B_sel[i], f"{B_sel[i]:.2f}", color=palette[i], fontsize=11, va="center")

plt.title(f"Bernstein Basis Polynomials (degree n={n})")
plt.xlabel("t")
plt.ylabel("$B_i^n(t)$")
plt.grid(True)
plt.legend(loc="upper left", bbox_to_anchor=(1.02, 1.0))
plt.tight_layout()
plt.show()